### using static (non-trainable) glove vectors with convolutional layers to classify newsgroup_20 documents

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html


In [7]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, Dropout, Embedding, GlobalMaxPooling1D, GlobalAvgPool1D
from keras.models import Model,Sequential

from helpers import (files_helper, 
                     metrics_helper, 
                     tags_helper,
                     texts_helper)


In [2]:
SEED=np.random.randint(1,1000)
SEED

189

In [3]:
np.random.seed(SEED)

In [4]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10
TOKENIZER_FILTERS='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' # default keras filters

In [8]:
embeddings_index = files_helper.read_glove(d=EMBEDDING_DIM)

In [9]:
texts,labels_index,labels_0 = files_helper.read_20_newgroup_files()       

In [10]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                      filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [11]:
len(word_index),len(labels_index)

(174074, 20)

In [13]:
# [inverse_word_index[idx] for idx in sequences[0]]

In [14]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

labels_1 = to_categorical(np.asarray(labels_0))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
y_train = labels_2[:-num_validation_samples]
X_val = data[-num_validation_samples:]
y_val = labels_2[-num_validation_samples:]

data.shape,labels_2.shape

((19997, 1000), (19997, 20))

In [15]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [16]:
embedding_matrix.shape

(174075, 100)

In [19]:
model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False))

model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128,activation='relu'))
model.add(Dense(len(labels_index),activation='relu'))
model.compile(loss ='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['acc'])

In [20]:
%%time

model.fit(X_train,y_train, validation_data=(X_val, y_val),
         epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 21s - loss: 6.1551 - acc: 0.1360 - val_loss: 7.4600 - val_acc: 0.1653
Epoch 2/10
15998/15998 [==============================] - 17s - loss: 5.1782 - acc: 0.2364 - val_loss: 5.0797 - val_acc: 0.2698
Epoch 3/10
15998/15998 [==============================] - 17s - loss: 5.0554 - acc: 0.2727 - val_loss: 5.6529 - val_acc: 0.2848
Epoch 4/10
15998/15998 [==============================] - 17s - loss: 4.9022 - acc: 0.3070 - val_loss: 5.0703 - val_acc: 0.3246
Epoch 5/10
15998/15998 [==============================] - 17s - loss: 4.8723 - acc: 0.3500 - val_loss: 5.1072 - val_acc: 0.3733
Epoch 6/10
15998/15998 [==============================] - 17s - loss: 4.9791 - acc: 0.3740 - val_loss: 5.0102 - val_acc: 0.4081
Epoch 7/10
15998/15998 [==============================] - 17s - loss: 4.8867 - acc: 0.3912 - val_loss: 5.2434 - val_acc: 0.3848
Epoch 8/10
15998/15998 [==============================]